Ejemplos CC5213
==

A continuación se definen funciones básicas para los ejemplos.

In [ ]:
import sys
import os
import cv2
import numpy
import easygui

print ("Usando OpenCV {} Python {}.{}.{}".format(cv2.__version__, sys.version_info.major, sys.version_info.minor, sys.version_info.micro))

def mostrar_frame(window_name, imagen, valorAbsoluto = False, escalarMin0Max255 = False):
    if valorAbsoluto:
        imagen_abs = numpy.abs(imagen)
    else:
        imagen_abs = imagen
    if escalarMin0Max255:
        imagen_norm = cv2.normalize(imagen_abs, dst=None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    else:
        imagen_norm = imagen_abs
    cv2.imshow(window_name, imagen_norm)


def abrir_video(filename):
    if filename is None:
        filename = 0
    elif filename.isdigit():
        filename = int(filename)
    if isinstance(filename, int):
        print ("abriendo camara {}".format(filename))
        capture = cv2.VideoCapture(filename)
    if (os.path.isfile(filename)):
        print ("abriendo archivo {}".format(filename))
        capture = cv2.VideoCapture(filename)
    if not capture.isOpened():
        raise Exception("no puedo abrir video {}".format(filename))
    return capture;


Ejemplo 1 - OTSU
==


In [ ]:
def ejemplo(filename):
    print ("abriendo {}".format(filename))
    imagen_color = cv2.imread(filename, cv2.IMREAD_COLOR)
    if imagen_color is None:
        print ("error abriendo {}".format(filename))
        return
    imagen_gris = cv2.cvtColor(imagen_color, cv2.COLOR_BGR2GRAY)
    threshold, imagen_bin = cv2.threshold(imagen_gris, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    window_name = os.path.basename(filename)
    cv2.imshow(window_name, imagen_color)
    cv2.imshow(window_name + " GRIS", imagen_gris)
    cv2.imshow(window_name + " BINARIA", imagen_bin)
    print ("{}: size={} threshold_otsu={}".format(window_name, imagen_color.shape, threshold))

filenames = easygui.fileopenbox(default="/", multiple=True)
if filenames is not None:
    for filename in filenames:
        ejemplo(filename)
    print ("Presione una tecla para salir...")
    cv2.waitKey(0)
    cv2.destroyAllWindows()


Ejemplo 2 - Sobel
==

In [ ]:
def ejemplo(filename, sobel_threshold):
    capture = abrir_video(filename)
    while capture.grab():
        retval, frame = capture.retrieve()
        if not retval:
            continue
        #convertir a gris
        frame_gris = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        cv2.imshow("VIDEO", frame_gris)
        #calcular filtro de sobel
        sobelX = cv2.Sobel(frame_gris, ddepth=cv2.CV_32F, dx=1, dy=0, ksize=3)
        sobelY = cv2.Sobel(frame_gris, ddepth=cv2.CV_32F, dx=0, dy=1, ksize=3)
        mostrar_frame("X", sobelX, valorAbsoluto=True, escalarMin0Max255=True)
        mostrar_frame("Y", sobelY, valorAbsoluto=True, escalarMin0Max255=True)
        #magnitud del gradiente
        magnitud = numpy.sqrt( numpy.square(sobelX) + numpy.square(sobelY) )
        mostrar_frame("MAGNITUD GRADIENTE", magnitud, escalarMin0Max255=True)
        #aproximacion de la magnitud del gradiente
        magnitud_aprox = numpy.abs(sobelX) + numpy.abs(sobelY)
        mostrar_frame("APROX GRADIENTE", magnitud_aprox, escalarMin0Max255=True)
        #umbral sobre la magnitud del gradiente
        retval, bordes = cv2.threshold(magnitud, thresh=sobel_threshold, maxval=255, type=cv2.THRESH_BINARY)
        retval, bordes_aprox = cv2.threshold(magnitud_aprox, thresh=sobel_threshold, maxval=255, type=cv2.THRESH_BINARY)
        mostrar_frame("BORDES", bordes)
        mostrar_frame("BORDES APROX", bordes_aprox)
        #esperar por una tecla
        key = cv2.waitKey(1) & 0xFF
        if key == ord(' '):
            key = cv2.waitKey(0) & 0xFF
        if key == ord('q') or key == 27:
            break
    capture.release()
    cv2.destroyAllWindows()

sobel_threshold = 80
filename = easygui.fileopenbox(default="/", multiple=False)

ejemplo(filename, sobel_threshold)


Ejemplo 3 - CANNY
==

In [ ]:
def ejemplo(filename, canny_threshold_1, canny_threshold_2):
    capture = abrir_video(filename)
    while capture.grab():
        retval, frame = capture.retrieve()
        if not retval:
            continue
        #convertir a gris
        frame_gris = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        cv2.imshow("VIDEO", frame_gris)
        #calcular canny
        frame_canny = cv2.Canny(frame_gris, threshold1=canny_threshold_1, threshold2=canny_threshold_2)
        cv2.imshow("CANNY", frame_canny)
        #esperar por una tecla
        key = cv2.waitKey(1) & 0xFF
        if key == ord(' '):
            key = cv2.waitKey(0) & 0xFF
        if key == ord('q') or key == 27:
            break
    capture.release()
    cv2.destroyAllWindows()

canny_threshold_1 = 50
canny_threshold_2 = 200
filename = easygui.fileopenbox(default="/", multiple=False)

ejemplo(filename, canny_threshold_1, canny_threshold_2)


Ejemplo 4 - Difference of Gaussians
==

In [ ]:
def ejemplo(filename, sigma1, sigma2, threshold):
    capture = abrir_video(filename)
    while capture.grab():
        retval, frame = capture.retrieve()
        if not retval:
            continue
        #convertir a gris
        frame_gris = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        cv2.imshow("VIDEO", frame_gris)
        #calcular DoG
        blur1 = cv2.GaussianBlur(frame_gris, (sigma1, sigma1), 0)
        blur2 = cv2.GaussianBlur(frame_gris, (sigma2, sigma2), 0)
        frame_diff = cv2.subtract(blur1, blur2)
        mostrar_frame("Diff", frame_diff, escalarMin0Max255=True)
        th, frame_bin = cv2.threshold(frame_diff, threshold, 255, cv2.THRESH_BINARY)
        mostrar_frame("BIN", frame_bin, escalarMin0Max255=True)
        #esperar por una tecla
        key = cv2.waitKey(1) & 0xFF
        if key == ord(' '):
            key = cv2.waitKey(0) & 0xFF
        if key == ord('q') or key == 27:
            break
    capture.release()
    cv2.destroyAllWindows()

sigma1 = 5
sigma2 = 13
threshold = 4
filename = easygui.fileopenbox(default="/", multiple=False)

ejemplo(filename, sigma1, sigma2, threshold)
